In [1]:
import sys
sys.path.append('/home/apple/anaconda3/envs/GeoLife/lib/python3.8/site-packages/')
sys.path

['/home/apple/GeoLife/Music_Generation',
 '/home/apple/anaconda3/lib/python38.zip',
 '/home/apple/anaconda3/lib/python3.8',
 '/home/apple/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/home/apple/anaconda3/lib/python3.8/site-packages',
 '/home/apple/anaconda3/lib/python3.8/site-packages/pyaml-21.10.1-py3.8.egg',
 '/home/apple/anaconda3/lib/python3.8/site-packages/IPython/extensions',
 '/home/apple/.ipython',
 '/home/apple/anaconda3/envs/GeoLife/lib/python3.8/site-packages/']

In [2]:
from midi_dataloader_v1 import MIDI_dataset_v1
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from collections import OrderedDict
from torchsummary import summary

In [3]:
TRAIN_PATH = './trainset'
VAL_PATH = './valset'
TEST_PATH = './testset'
BATCH = 1
WORKER = 20

In [4]:
train_dataset = MIDI_dataset_v1(TRAIN_PATH)
val_dataset = MIDI_dataset_v1(VAL_PATH)
test_dataset = MIDI_dataset_v1(TEST_PATH)

In [5]:
trainloader = DataLoader(train_dataset,batch_size=BATCH,shuffle=True,num_workers=WORKER)
valloader = DataLoader(val_dataset,batch_size=BATCH,num_workers=WORKER)
testloader = DataLoader(test_dataset,batch_size=BATCH,num_workers=WORKER)

In [6]:
inputs = next(iter(trainloader))
print(inputs.shape)

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/apple/anaconda3/envs/GeoLife/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/apple/anaconda3/envs/GeoLife/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/apple/anaconda3/envs/GeoLife/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/apple/GeoLife/Music_Generation/midi_dataloader_v1.py", line 76, in __getitem__
    mid = MidiFile(self.file_list[idx])
  File "/home/apple/anaconda3/lib/python3.8/site-packages/mido/midifiles/midifiles.py", line 315, in __init__
    with io.open(filename, 'rb') as file:
FileNotFoundError: [Errno 2] No such file or di

In [7]:
class CRNN(nn.modules.Module):
    def __init__(self):
        super(CRNN,self).__init__()
        
        # N Freq Time Channel
        self.freqpad = nn.ZeroPad2d((0, 0, 37, 37))
        self.bn0freq = nn.BatchNorm2d(num_features=96)
        
        # N Channel Freq Time
        self.cnn = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1, 64, kernel_size=(3, 3), padding=(1, 1), bias=False)),
          ('bn1', nn.BatchNorm2d(num_features=64)),
          ('elu1', nn.ELU()),
          ('pool1', nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))),
          ('dropout1', nn.Dropout(p=0.1)),
          ('conv2', nn.Conv2d(64, 128, kernel_size=(3, 3), padding=(1, 1), bias=False)),
          ('bn2', nn.BatchNorm2d(num_features=128)),
          ('elu2', nn.ELU()),
          ('pool2', nn.MaxPool2d(kernel_size=(3, 3), stride=(3, 3))),
          ('dropout2', nn.Dropout(p=0.1)),
          ('conv3', nn.Conv2d(128, 128, kernel_size=(3, 3), padding=(1, 1), bias=False)),
          ('bn3', nn.BatchNorm2d(num_features=128)),
          ('elu3', nn.ELU()),
          ('pool3', nn.MaxPool2d(kernel_size=(4, 4), stride=(4, 4))),
          ('dropout3', nn.Dropout(p=0.1)),
          ('conv4', nn.Conv2d(128, 128, kernel_size=(3, 3), padding=(1, 1), bias=False)),
          ('bn4', nn.BatchNorm2d(num_features=128)),
          ('elu4', nn.ELU()),
          ('pool4', nn.MaxPool2d(kernel_size=(4, 4), stride=(4, 4))),
          ('dropout4', nn.Dropout(p=0.1)),
        ]))
        
        self.lstm1 = nn.LSTM(128, 32)
        self.lstm2 = nn.LSTM(32, 32)
        self.dropout = nn.Dropout(p=0.3)
        
        self.fc = nn.Linear(32, 11)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        x = self.freqpad(x)
        x = self.bn0freq(x)

        x = x.permute(0, 3, 1, 2)
        x = self.cnn(x)
        
        x = x.squeeze()
        x = x.permute(0, 2, 1)
        
        # rnn
        x, (h_0, c_0) = self.lstm1(x)
        x, (h_1, c_1) = self.lstm2(x, (h_0, c_0))
        x = x[:,-1,:]
        
        x = self.dropout(x)
        x = self.fc(x)
        x = self.sigmoid(x)
        
        return x

In [8]:
x = torch.randn(2, 96, 1366, 1)
model = CRNN()
model.to('cpu')
print(model)

y = model(x)
print(y.shape)

CRNN(
  (freqpad): ZeroPad2d((0, 0, 37, 37))
  (bn0freq): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cnn): Sequential(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (elu1): ELU(alpha=1.0)
    (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (dropout1): Dropout(p=0.1, inplace=False)
    (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (elu2): ELU(alpha=1.0)
    (pool2): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
    (dropout2): Dropout(p=0.1, inplace=False)
    (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(128, eps=1e-05, mo